<a href="https://colab.research.google.com/github/ANUPRIYAR/Sentiment_Analysis_of_IMDB_MovieReviews/blob/main/IMDB_movie_review_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# !unzip archive.zip

In [85]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

In [69]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [70]:
path = '/content/IMDB Dataset.csv'
df = pd.read_csv(path)

In [71]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [72]:
# Map positive to 1 and negaitve to 0 in the Dataframe
df['sentiment'] = df.sentiment.map({'positive': 1 , 'negative' : 0})

In [73]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [74]:
# We create a new column called kfold
df['kfold'] = -1

In [75]:
# Randomize the rows of data
df = df.sample(frac=1).reset_index(drop=True)

In [76]:
# Fetch Labels
y = df.sentiment.values

In [77]:
# Initiate the kfold class from model_slection module
kf = model_selection.StratifiedKFold(n_splits=5)

In [78]:
# fill the new kfold col
for f, (t_,v_) in enumerate(kf.split(X=df, y=y)):
  df.loc[v_,'kfold'] = f


In [79]:
df.head()

,review,sentiment,kfold
0,This is a textbook example of how Hollywood di...,0,0
1,This is an interesting movie. I think it's ver...,1,0
2,This is one of those movies you think that the...,0,0
3,I had borrowed the three Sarah movies from a f...,1,0
4,Usually I do not like movies with/about aliens...,1,0


In [80]:
df['kfold'].value_counts()

4    10000
3    10000
2    10000
1    10000
0    10000
Name: kfold, dtype: int64

In [86]:
# we go over the fold created
for fold_ in range(5):
  # temprory dataframes for train and test
  train_df = df[df.kfold != fold_].reset_index(drop=True)
  test_df = df[df['kfold'] == fold_].reset_index(drop=True)


  # Intialize CountVectorizer with NLTK's word_tokenize function as tokenizer
  count_vec = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)

  # fit count_vec on training data reviews
  count_vec.fit(train_df.review)

  # transform and training and validation data reviews
  xtrain = count_vec.transform(train_df.review)
  xtest = count_vec.transform(test_df.review)


  # Initialize Logistic Regression model
  model  = linear_model.LogisticRegression()
  model.fit(xtrain,train_df.sentiment)


  # Make predictions on test data reviews
# Threshold for prediction is 0.5
  preds = model.predict(xtest)

  accuracy = metrics.accuracy_score(test_df.sentiment, preds)
  print(f"Fold: {fold_}")
  print(f"Accuracy: {accuracy}")

Fold: 0
Accuracy: 0.8902
Fold: 1
Accuracy: 0.8933
Fold: 2
Accuracy: 0.895
Fold: 3
Accuracy: 0.8932
Fold: 4
Accuracy: 0.8924
